In [1]:
import sqlite3
import time
import os
import psycopg2

import sys
sys.path.append(r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\ccf_slice_registration\pipeline')
from ccf_slice_registration_functions import *


In [3]:
ccf_reg_root_directory = r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\ccf_slice_registration\pipeline\ccf_reg_output'
database_file = r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\ccf_slice_registration\ccf_reg_database\ccf_reg.db'

In [4]:
con = sqlite3.connect(database_file)
cur = con.cursor()


In [5]:

# check that the registered_slices and registered_cells tables exists
res = cur.execute("SELECT name FROM sqlite_master")
if ("registered_slices",) not in res.fetchall():
    cur.execute(
        "CREATE TABLE registered_slices(slice_id, slice_name, registered, overview_to_virtual_slice_transform_path, virtual_slice_to_ccf_transform_path, registration_timestamp)")

res = cur.execute("SELECT name FROM sqlite_master")
if ("registered_cells",) not in res.fetchall():
    cur.execute(
        "CREATE TABLE registered_cells(specimen_id, specimen_name, slice_id, has_manual, manual_registered, manual_swc_path, manual_registration_timestamp, has_raw_autotrace, raw_autotrace_registered, raw_autotrace_swc_path, raw_autotrace_registration_timestamp, has_post_processed_step_14_autotrace, post_processed_step_14_autotrace_registered, post_processed_step_14_autotrace_swc_path, post_processed_step_14_autotrace_registration_timestamp, has_post_processed_step_22_autotrace, post_processed_step_22_autotrace_registered, post_processed_step_22_autotrace_swc_path, post_processed_step_22_autotrace_registration_timestamp)")


In [6]:
reg_slices = cur.execute("SELECT * FROM registered_slices")
existing_slice_registrations = reg_slices.fetchall()
reg_cells = cur.execute("SELECT * FROM registered_cells")
existing_cell_registrations = reg_cells.fetchall()
package_list = []

cur.close()
con.close()

In [10]:
slice_package_list = []
cell_package_list = []
for slice_name in os.listdir(ccf_reg_root_directory):
    #TODO do some validation it's a good slice name (try to get slice id and if it doesn't work it's not a valid slice name?)
    slice_id = get_id_by_name(slice_name)

    #check if slice has been transformed 
    slice_is_registered = 0
    slice_registered_time = 0
    #slice has both transform files?
    overview_to_virtual_transform_path = os.path.join(ccf_reg_root_directory, slice_name, 'overview_to_virtual_slice_transform.txt')
    virtual_to_ccf_transform_path = os.path.join(ccf_reg_root_directory, slice_name, 'virtual_slice_to_ccf_transform.txt')
    if os.path.isfile(overview_to_virtual_transform_path):
        if os.path.isfile(virtual_to_ccf_transform_path):
            slice_is_registered = 1
            slice_registered_time = os.path.getctime(virtual_to_ccf_transform_path)
            slice_registered_time = time.ctime(slice_registered_time)

    #check if any cells have been registered
    swc_dir = os.path.join(ccf_reg_root_directory, slice_name, 'SWC')
    swc_files = []
    if os.path.exists(swc_dir):
        swc_files = [f for f in os.listdir(swc_dir) if f.endswith("_reoriented_pir_shifted.swc")]

    if swc_files:
        #get list of all cells with swcs 
        cell_list = list(set([file.split('_', 1)[0] for file in swc_files]))

        for cell_id in cell_list:
            #determine the swc registration for each cell 
            cell_swcs = [file for file in swc_files if file.startswith(cell_id)]

            cell_name = get_name_by_id(cell_id)

            #does this cell have a manual to reconstruct?
            has_manual = 0
            manual_registered = 0
            manual_swc_path = "None"
            manual_timestamp = "None"
            try: lims_path = list(get_swc_from_lims(str(cell_id)))[1]
            except: lims_path = None
            if lims_path:
                has_manual = 1
                #has manual been reconstructed?
                manual_name = cell_id + '_reoriented_pir_shifted.swc'
                if manual_name in cell_swcs:
                    manual_registered = 1
                    manual_swc_path = os.path.join(swc_dir, manual_name)
                    manual_registration_timestamp = os.path.getctime(manual_swc_path)
                    manual_registration_timestamp = time.ctime(manual_registration_timestamp)

            #does this cell have a raw autotrace to reconstruct?
            has_raw_auto = 0
            raw_auto_registered = 0
            raw_auto_swc_path = "None"
            raw_auto_timestamp = "None"
            raw_path = get_autotrace_raw_path(cell_id)
            if os.path.isfile(raw_path):
                has_raw_auto = 1
                #has manual been reconstructed?
                raw_auto_name = cell_id + '_autotrace_raw_reoriented_pir_shifted.swc'
                if raw_auto_name in cell_swcs:
                    raw_auto_registered = 1
                    raw_auto_swc_path = os.path.join(swc_dir, raw_auto_name)
                    raw_auto_timestamp = os.path.getctime(raw_auto_swc_path)
                    raw_auto_timestamp = time.ctime(raw_auto_timestamp)

            #does this cell have a post processed step 14 autotrace to reconstruct?
            has_pp14_auto = 0
            pp14_auto_registered = 0
            pp14_auto_swc_path = "None"
            pp14_auto_timestamp = "None"
            pp14_path = get_autotrace_pp_path(cell_id, 14)
            if os.path.isfile(pp14_path):
                has_pp14_auto = 1
                #has manual been reconstructed?
                pp14_auto_name = cell_id + '_autotrace_pp14_reoriented_pir_shifted.swc'
                if pp14_auto_name in cell_swcs:
                    pp14_auto_registered = 1
                    pp14_auto_swc_path = os.path.join(swc_dir, pp14_auto_name)
                    pp14_auto_timestamp = os.path.getctime(pp14_auto_swc_path)
                    pp14_auto_timestamp = time.ctime(pp14_auto_timestamp)

            #does this cell have a post processed step 22 autotrace to reconstruct?
            has_pp22_auto = 0
            pp22_auto_registered = 0
            pp22_auto_swc_path = "None"
            pp22_auto_timestamp = "None"
            pp22_path = get_autotrace_pp_path(cell_id, 22)
            if os.path.isfile(pp22_path):
                has_pp22_auto = 1
                #has manual been reconstructed?
                pp22_auto_name = cell_id + '_autotrace_pp22_reoriented_pir_shifted.swc'
                if pp22_auto_name in cell_swcs:
                    pp22_auto_registered = 1
                    pp22_auto_swc_path = os.path.join(swc_dir, pp22_auto_name)
                    pp22_auto_timestamp = os.path.getctime(pp22_auto_swc_path)
                    pp22_auto_timestamp = time.ctime(pp22_auto_timestamp)
            
            #collect cell info to save to db if it's a new entry 
            cell_package = (int(cell_id), cell_name, int(slice_id), 
                            has_manual, manual_registered, manual_swc_path, manual_registration_timestamp, 
                            has_raw_auto, raw_auto_registered, raw_auto_swc_path, raw_auto_timestamp, 
                            has_pp14_auto, pp14_auto_registered, pp14_auto_swc_path, pp14_auto_timestamp, 
                            has_pp22_auto, pp22_auto_registered, pp22_auto_swc_path, pp22_auto_timestamp)
            if cell_package not in existing_cell_registrations:
                cell_package_list.append(cell_package)

    #collect slice info to save to db if it's a new entry 
    if not slice_is_registered: 
        slice_registered_time = "None"
        overview_to_virtual_transform_path = "None"
        virtual_to_ccf_transform_path = "None"
    slice_package = (int(slice_id), slice_name, slice_is_registered, overview_to_virtual_transform_path, virtual_to_ccf_transform_path, slice_registered_time)

    if slice_package not in existing_slice_registrations:
        slice_package_list.append(slice_package)



In [11]:
#save new slice registration info to db
if slice_package_list:
    con = sqlite3.connect(database_file)
    cur = con.cursor()

    insert_string = " \n".join([f"({p[0]}, '{p[1]}', {p[2]}, '{p[3]}', '{p[4]}', '{p[5]}')," for p in slice_package_list])[:-1]  # last comma
    insert_cmd = f"""
        INSERT INTO registered_slices VALUES
        {insert_string}
    """
    cur.execute(insert_cmd)
    con.commit()

    cur.close()
    con.close()


In [12]:
#save new cell registration info to db
if cell_package_list:
    con = sqlite3.connect(database_file)
    cur = con.cursor()

    insert_string = " \n".join([f"({p[0]}, '{p[1]}', {p[2]}, {p[3]}, {p[4]}, '{p[5]}', '{p[6]}',{p[7]}, {p[8]}, '{p[9]}', '{p[10]}',{p[11]}, {p[12]}, '{p[13]}', '{p[14]}',{p[15]}, {p[16]}, '{p[17]}', '{p[18]}')," for p in cell_package_list])[:-1]  # last comma
    
    insert_cmd = f"""
        INSERT INTO registered_cells VALUES
        {insert_string}
    """
    cur.execute(insert_cmd)
    con.commit()

    cur.close()
    con.close()